# ***projekt numerik***

In [9]:
import numpy as np
from matplotlib import pyplot as plt
import constants as cst

## ***Implicit model***

In [10]:
# delta's discretisation

delta_t = 0.5
delta_r = 0.5
n = 4

def t_i(i):
    return i*delta_t

def r_j(j):
    return j*delta_r

#T_i_plus_one = np.zeros((n-1,n-1))


##  Constants and their implementations

blablabla

In [11]:
#Dichte nach Radius

def rho(r):

    #dichte;

    if(r>= 0 or r <= 25):
        dichte = cst.WASSER[0]

    if((r>= 25 or r <= 30) or (r>= 55 or r<=60) ):
        dichte =cst.STAHL[0]

    if(r>= 30 or r <= 55):
        dichte = cst.GLASSWOLLE[0]

    return dichte

# Wärmekapazität nach Radius
def c_p(r):

    #cap;

    if(r>= 0 or r <= 25):
        cap = cst.WASSER[1]

    if((r>= 25 or r <= 30) or (r>= 55 or r<=60)):
        cap = cst.STAHL[1]

    if(r>= 30 or r <= 55):
        cap = cst.GLASSWOLLE[1]

    return cap


#Wärmeleitungskonstante nach Radius
def Lambda(r):

    #lam;

    if(r>= 0 or r <= 25):
        lam = cst.WASSER[2]

    if((r>= 25 or r <= 30) or (r>= 55 or r<=60) ):
        lam = cst.STAHL[2]

    if(r>= 30 or r <= 55):
        lam = cst.GLASSWOLLE[2]

    return lam

## Definition of the function of time and its implementation



In [12]:
# function definition for explicit solution
'''

def T_i_p1(T_i,n):

    t_i_p1 = []

    # add to T_i,j the follow up values, iterating on the value J=1 to J = n-1
    for j in range (1, n - 1):
    # first and second derivatives respective to r
        T_prime = T_i[j+1]-T_i[j]/(delta_r)
        T_prime_2 = T_i[j+1]-2*T_i[j]+T_i[j-1]/(delta_r**2)

        t_i_p1.append(T_i[j] + (delta_t*Lambda(j)) / (r_j(j) * c_p(r_j(j)) * rho(r_j(j))) * (T_prime + r_j(j) * T_prime_2)) 
    
    return t_i_p1
'''

'\n\ndef T_i_p1(T_i,n):\n\n    t_i_p1 = []\n\n    # add to T_i,j the follow up values, iterating on the value J=1 to J = n-1\n    for j in range (1, n - 1):\n    # first and second derivatives respective to r\n        T_prime = T_i[j+1]-T_i[j]/(delta_r)\n        T_prime_2 = T_i[j+1]-2*T_i[j]+T_i[j-1]/(delta_r**2)\n\n        t_i_p1.append(T_i[j] + (delta_t*Lambda(j)) / (r_j(j) * c_p(r_j(j)) * rho(r_j(j))) * (T_prime + r_j(j) * T_prime_2)) \n    \n    return t_i_p1\n'

In [13]:
Gt=20
n = int(cst.size_rhor/delta_r)
time =int(100 / delta_t)

#anfangsbedigung (2a) t(0,r) = 0

T_initial = np.zeros(n-1) # potential rounding error that can happen here, be careful with the delta_r division increments. it should always be a multiple of 5,3 and 2.

# randbedigung 1 (2b) T(t,R) = g(t)
T_R1 = np.ones(n-1)*Gt


## explicit

$T_{i+1,j} = u_j \frac{r_j}{\Delta r^2}T_{i,j-1} + [1-u_j(\frac{1}{\Delta r} + \frac{2r_j}{\Delta r^2})]T_{i,j} + u_j(\frac{1}{\Delta r}+ \frac{r_j}{\Delta r^2})T_{i,j+1}$


for $u_j = \frac{\Delta t}{r_j}k_j$ and $k_j = \frac{\lambda _j}{Cp_{j} \rho _j}$

In [23]:

T = np.empty([n-1,n-1])


for i in range(0,time):
    for j in range (1,n-1):

        k_j = Lambda(j) / (c_p(j)*rho(j))
        u_j = (delta_t*k_j) / r_j(j)

        #boundary case where we have to evaluate Ti,j-1 where j-1 = 0
        if i == 0:
            A,B,C = 0,0,0
        else:
            A = T[i][j-1]
            B = T[i][j]
        #boundary case where we have to evaluate Ti,j+1 where j+1 = R
        if j+1 == n-1:
            C = Gt
        else:  
            C = T[i][j+1]
    
        T[i][j] = u_j*r_j(j)/(delta_r**2)*A + (1-u_j*(1/delta_r + 2*r_j(j)/delta_r**2))*B + u_j*(1/delta_r + r_j(j)/delta_r**2)*C
        


IndexError: index 119 is out of bounds for axis 0 with size 119